# Configuration

Before using nextnanopy, one has to configure it. Configuration are explained in details [here](https://github.com/nextnanopy/nextnanopy/blob/master/docs/examples/Example0_Set_up_the_configuration.ipynb) . If you have not done it yet, please, fill and run the cell below. You can use nextnanomat to find the paths

*Tools->Options->Simulation->nextnano++*

*Tools->Options->licenses->nextnano++*

*Tools->Options->Material database->nextnano++*

In [ ]:
import nextnanopy as nn
"""Uncomment and fill these lines to config nextnanopy"""
#nn.config.set('nextnano++','exe',r'path to nextnano++ exe file')
#nn.config.set('nextnano++','license',r'path to nextnano++ license file')
#nn.config.set('nextnano++','database',r'path to nextnano++ database file')
#nn.config.set('nextnano++','outputdirectory',r'full path to directory where you want to save output')
#nn.config.save()

###  Import necessary packages

If you do not have some of this packages, open Anaconda Prompt and run a command.

*pip install package_name*

and then run again a cell with import statements

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import nextnanopy as nn
import os
import re
import natsort
palette = ['#343131','#13adb5']
plt.style.use('default')
plt.style.use('seaborn-deep')
%matplotlib inline

In [ ]:
software = 'nextnano++'
nn.config.config[software]

### Run an input file and make a preview plot


In [ ]:
filepath = r'Superlattice_N_Wells.in'
input_file = nn.InputFile(filepath)
input_file.execute()


In [ ]:
output_datafolder = nn.DataFolder(input_file.folder_output)
bandedges_filepath = output_datafolder.go_to('bias_00000','bandedges.dat')
bandedges_datafile = nn.DataFile(bandedges_filepath)
figure, axes = bandedges_datafile.plot()
axes.legend(bbox_to_anchor=(1.04,1), loc="upper left")


In case you do not know the name of the file you are looking for, use nextnanopy.DataFolder.show_tree() feature to see all files in the output

In [ ]:
output_datafolder.show_tree()

## TO DO plot the hydrostatic strain

In [ ]:
### YOUR CODE HERE

strain_datafile.plot()

### Fine tuning of the plot


In [ ]:
x = bandedges_datafile.coords['x'].value
gamma = bandedges_datafile.variables['Gamma'].value
bandedgeoffset = np.amin(gamma[30:-30]) #Values chosen to avoid the 0 value at the start and end
bandedgeoffset
gamma_shift = (gamma - bandedgeoffset)*1000

fig, axes = plt.subplots()
axes.plot(x, gamma_shift, c = 'black')
axes.set_ylim(0,400)

wf_file = output_datafolder.go_to('bias_00000', 'Quantum', 'amplitudes_shift_quantum_region_Gamma_00000.dat')
wf_datafile = nn.DataFile(wf_file)
gamma1_wf = (wf_datafile.variables['Psi_1'].value - bandedgeoffset)*1000
x_wf = wf_datafile.coords['x'].value
axes.plot(x_wf,gamma1_wf)




## TO DO: make the same plot as above, but for heavy hole

In [ ]:
###YOUR CODE HERE

fig, axes = plt.subplots()
axes.plot(x, hh_bandedge, c = 'black')
axes.plot(x_hh,heavy_hole_wf)


## Changing variables in input file

We want to change
* <code>WELl_WIDTH</code>
* <code>BARRIER_WIDTH</code>
* <code>LEFT_BARRIER_WIDTH</code>
* <code>RIGHT_BARRIER_WIDTH</code>
* <code>NUMBER_OF_WELLS</code>



In [ ]:
input_file.set_variable(name = "WELL_WIDTH", value = 10)
input_file.set_variable(name = "BARRIER_WIDTH", value = 10)
input_file.set_variable(name = "LEFT_BARRIER_WIDTH" , value = 10)
input_file.set_variable(name = "RIGHT_BARRIER_WIDTH" , value = 10)
#input_file.set_variable(name = "NUMBER_OF_WELLS", value = 4)


#my_input_file_new = os.path.join(output_folder, r'Superlattice_N_Wells_WF2' + ".in")
my_input_file_new = 'Multiple_Quantum_Well.in'
input_file.save(my_input_file_new, overwrite = True)

In [ ]:
input_file.execute()

In [ ]:
output_datafolder = nn.DataFolder(input_file.folder_output)
bandedges_filepath = output_datafolder.go_to('bias_00000','bandedges.dat')
bandedges_datafile = nn.DataFile(bandedges_filepath)
x = bandedges_datafile.coords['x'].value
gamma = bandedges_datafile.variables['Gamma'].value
bandedgeoffset = np.amin(gamma[30:-30]) #Values chosen to avoid the 0 value at the start and end
bandedgeoffset
gamma_shift = (gamma - bandedgeoffset)*1000

fig, axes = plt.subplots()
axes.plot(x, gamma_shift, c = 'black')
axes.set_ylim(0,300)

wf_file = output_datafolder.go_to('bias_00000', 'Quantum', 'amplitudes_shift_quantum_region_Gamma_00000.dat')
wf_datafile = nn.DataFile(wf_file)
gamma1_wf = (wf_datafile.variables['Psi_1'].value - bandedgeoffset)*1000
x_wf = wf_datafile.coords['x'].value
axes.plot(x_wf,gamma1_wf)

## TODO: Make the barrier width 3 times the well width and plot the result for lowest electron state

In [ ]:
### YOUR CODE HERE


axes.plot(x_wf,gamma1_wf)

## Sweeps 

How to figure out how many quantum wells are needed for system to behave as "infinite" quantum well supercell. For this we can observe how energy changes with the number of wells. 

For more details about sweeps, refer to [tutorial on Sweep](https://github.com/nextnanopy/nextnanopy/blob/master/docs/examples/Example4_Sweep_to_automate_the_execution.ipynb)

In [ ]:
SweepVariable = 'NUMBER_OF_WELLS'
SweepValues = np.arange(2,20,2)

SweepingVariables = {SweepVariable: SweepValues}
#We can always increase the size of the dictionary if we have
#more variables we want to sweep over.

In [ ]:
sweep = nn.Sweep(SweepingVariables, filepath)
sweep.save_sweep()

In [ ]:
sweep.execute_sweep()

In [ ]:
sweep.sweep_output_directory

In [ ]:
sweepFolder = nn.DataFolder(sweep.sweep_output_directory)

In [ ]:
sweepFolder.show_tree()

In [ ]:
sweep_subfolders = natsort.natsorted(sweepFolder.folders.keys())

def get_all_energies(sweepDataFolder, offset = bandedgeoffset):
    """
    return all ground state energies calculated from sweep in meV
    """
    arr = np.array([])
    folders = natsort.natsorted(sweepDataFolder.folders.keys())
    for folder in folders:
        energy = (nn.DataFile(sweepDataFolder.go_to(folder,'bias_00000', 'Quantum','amplitudes_shift_quantum_region_Gamma_00000.dat'), product = 'nextnano++').variables['E_1'].value- offset) * 1000
        energy = np.amin(energy)
        arr = np.append(arr,energy)
    return arr



In [ ]:
groundstate_energies = get_all_energies(sweepFolder)
print(groundstate_energies)


In [ ]:
fig, ax = plt.subplots()
ax.plot(SweepValues, groundstate_energies, 'o--', color = palette[1])
ax.grid(visible = True)
ax.set_ylabel('Energy (meV)')
ax.set_xlabel('Number of wells')
ax.set_title('Ground state energies')


## TO DO: plot the wavefunction vs bandedge for the supercell with 18 wells.

In [ ]:
wf1 = (nn.DataFile(sweepFolder.go_to(sweep_subfolders[-1], 
         'bias_00000', 'Quantum','amplitudes_shift_quantum_region_Gamma_00000.dat'), 
            product = software).variables['Psi_1'].value - bandedgeoffset) * 1000

bandedge10 = ### YOUR CODE HERE

x = ### YOUR CODE HERE 

xwf1 = ### YOUR CODE HERE

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, bandedge10, color = palette[0])
ax.plot(xwf1, wf1, color = palette[1])

ax.set_ylim(0,300)
ax.set_xlabel('Distance (nm)')
ax.set_ylabel('Energy (meV)')
ax.set_title('Finite Superlattice')


## TODO plot the ground state energies vs number of wells for the well width = 3 nm, barrier width = 3 nm.

In [ ]:
###your code here

In [ ]:
fig, ax = plt.subplots()
ax.plot(SweepValues, groundstate_energies, 'o--', color = palette[1])
ax.grid(visible = True)
ax.set_ylabel('Energy (meV)')
ax.set_xlabel('Number of wells')
ax.set_title('Ground state energies')


## TODO plot the ground state energies vs well_width for a system with 5 wells, width between 2 and 20 nm.

In [ ]:
### YOUR CODE HERE


## TODO (optional) Imagine you look for a combination of well width and barrier width that gives given confinement energy.
Let N_WELLS = 4. Barrier width and well width can be between 2 and 10 nm with a step 1 nm. (all combination possible). Find the combination whith the confinement energy of ground electron state as close as possible to 77 mEv.
You may find this function useful:

*nextnanopy.postprocess.simple_optimize*